<a href="https://colab.research.google.com/github/Zamoca42/TIL/blob/main/Python/Django_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로젝트 소개

- URL Shrotener
  - Front-end 파트보다 더 중요한 비지니스 로직에 집중
  - 복잡한 통계 데이터 쿼리 능력 배양
  - 클라이언트 정보를 알아내는 기법
  - 각종 마이너 테크닉 습득 가능
  - 단순 리다이렉팅이 아니라, 통계, 실제 휴먼 트래픽 여부 등 확인
  - 특정 환경에서의 페이지 접근 통제

# 프로젝트 Tasks

## Front-end
  - 메인 랜딩 페이지
  - 로그인/회원가입 페이지 
  - 비밀번호 찾기 페이지
  - 이메일 인증
  - URL 단축
  - 통계 페이지
  - 회원개인프로필설정
  - 회사설정
  - Vue.js

## Back-end
- 정책수립
- DB 모델링
- Rendering Views
- API 만들기
- Cache 사용
- Django ORM 이용한 쿼리


## 기타
- 텔레그램 봇 만들기 
- Static File 관리
- CICD 구성

# 정책

## 기본
  - User당50개생성가능
  - 리프레시 하지 않으면 60일 유효
  - 이메일 미인증시 사용 불가
  - 1초에같은IP에서5회이상호출불가 - 기본통계 제공
  - 302리턴
  
## 유료
  - 무제한 생성 가능
  - 삭제할때까지 삭제되지 않음
  - 이메일 미인증시 사용 불가
  - 1초에같은IP에서20회이상호출불가 - 어드벤스드 통계 제공
  - 301리턴

# Rendering Views, Json Response Views

## 1. HTTPResponse

- Rendering View, Json Response View 모두 API의 일종
- HTML을 렌더링 하는지, Json의 형태로 단순히 정보만을 주는 차이
- 렌더링을 해야할 때는 Django의 render 를 사용 (HTTPResponse 사용)
- Json 포맷으로 전달해야 할 때는 JsonResponse를 사용

## 2. RestAPI

### REST(REpresentational State Transfer)
- 프로토콜 아님 
- 표준 아님

### 특징
- Uniform
- Stateless
- Cacheable

### 구현
- Method로 하는 행위 구별 
- Item과 id만 URL에 사용
- “-”는가독성을위해사용 
- “_” 는 사용하지 않음
- 파일 확장자는 uri에 포함하지 않음 
- 언제나 소문자만 사용

### 예시
- GET http://abc.com/users
- GET http://abc.com/user/1
- POST http://abc.com/users
- PUT http://abc.com/user/1
- DELETE http://abc.com/user/1
- GET http://abc.com/user/1/todos

# DB Modelling

## settings.py

In [ ]:
# Database
# https://docs.djangoproject.com/en/3.2/ref/settings/#databases

DATABASES = {
    "default": {
        "ENGINE": "django.db.backends.mysql",
        "NAME": "app_db",
        "USER": "root",
        "PASSWORD": "Xptmxm123!",
        "HOST": "34.64.241.134",
        "PORT": 3306,
        "OPTIONS": {"autocommit": True, "charset": "utf8mb4"},
    }
}

## 1. User, Org, PayPlan

In [ ]:
class PayPlan(models.Model):
  name = models.CharField(max_length=20)
  price = models.IntegerField()
  updated_at = models.DateTimeField(auto_now=True) 
  create_at = models.DateTimeField(auto_now_add=True)

class Organization(models.Model):
   class Industries(models.TextChoices):
      PERSONAL = "personal"
      RETAIL = "retail"
      MANUFACTURING = "manufacturing"
      IT = "it"
      OTHERS = "others"
  name = models.CharField(max_length=50)
  industry = models.CharField(max_length=15, choices=Industries.choices, default=Industries.OTHERS) 
  pay_plan = models.ForeignKey(PayPlan, on_delete=models.DO_NOTHING, null=True)
  updated_at = models.DateTimeField(auto_now=True)
  create_at = models.DateTimeField(auto_now_add=True)
  
class Users(AbstractUser):
  full_name = models.CharField(max_length=100, null=True)
  organization = models.ForeignKey(Organization, on_delete=models.DO_NOTHING, null=True)

## 2. Email Verification

In [ ]:
class EmailVerification(models.Model):
  user = models.ForeignKey(Users, on_delete=models.CASCADE) 
  key = models.CharField(max_length=100, null=True) 
  verified = models.BooleanField(default=False)
  updated_at = models.DateTimeField(auto_now=True) 
  create_at = models.DateTimeField(auto_now_add=True)
class Categories(models.Model):
  name = models.CharField(max_length=100)
  organization = models.ForeignKey(Organization, on_delete=models.DO_NOTHING, null=True) 
  creator = models.ForeignKey(Users, on_delete=models.CASCADE)
  updated_at = models.DateTimeField(auto_now=True)
  created_at = models.DateTimeField(auto_now_add=True)

In [ ]:
class ShortenedUrls(models.Model):
    class UrlCreatedVia(models.TextChoices):
        WEBSITE = "web"
        TELEGRAM = "telegram"
    def rand_string():
      str_pool = string.digits + string.ascii_letters
      return ("".join([random.choice(str_pool) for _ in range(6)])).lower()

    nick_name = models.CharField(max_length=100)
    category = models.ForeignKey(Categories, on_delete=models.DO_NOTHING, null=True)
    prefix = models.CharField(max_length=50)
    creator = models.ForeignKey(Users, on_delete=models.CASCADE)
    target_url = models.CharField(max_length=2000)
    shortened_url = models.CharField(max_length=6, default=rand_string)
    create_via = models.CharField(max_length=8, choices=UrlCreatedVia.choices, default=UrlCreatedVia.WEBSITE) expired_at = models.DateTimeField(null=True)
    updated_at = models.DateTimeField(auto_now=True)
    created_at = models.DateTimeField(auto_now_add=True)

# Django Rest Framework

- DRF란?
  - Django안에서 RESTful API서버를 쉽게 구축할 수 있도록 도와주는 라이브러리
  - Micro Framework.


- Serializer
  - Json -> Python 객채 
  - Python 객체 -> Json  

    ![스크린샷 2023-02-04 오후 11 26 34](https://user-images.githubusercontent.com/96982072/216772901-4abeed15-46c3-4b3d-9bf7-d783a032113f.png)


- 설치

In [ ]:
$> pip install djangorestframework

- settings.py

In [ ]:
INSTALLED_APPS = [ 
    ...
    'rest_framework’, 
    ]